In [1]:
import pandas as pd
import os
import shutil
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [41]:
df = pd.read_csv("./Augmentation/test_df_names.csv")
from tensorflow.keras.models import load_model
model = load_model("0.7164(new).h5")
X = np.array(df.drop(["diagnostic", "img_id"], axis=1))
y = np.array(df["diagnostic"])
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

# Load images using ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 600
target_size = (224, 224)
class_mode = 'categorical'
image_dir = './zr7vgbcyr2-1/all/test'

train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_dir,
    x_col="img_id",
    y_col="diagnostic",
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    shuffle=True)
xtest, ytest = next(train_generator)


Found 40 validated image filenames belonging to 4 classes.


In [29]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from tensorflow.keras.applications.densenet import preprocess_input as preprocess_input_densenet
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_input_mobilenet
from tensorflow.keras.models import load_model
import os
import pandas as pd
model_names = ["resnet", "densenet", "mobilenet"]

def load_image(image_path, target_size=(224, 224)):
    """
    Loads an input image into PIL format of specified size
    """
    img = image.load_img(path=image_path, target_size=target_size)
    return img

def preprocess_image(loaded_image, model_name):
    """
    Preprocesses a loaded image based on the specified model
    """
    img_array = image.img_to_array(loaded_image)
    img_batch = np.expand_dims(img_array, axis=0)
    
    if model_name == "resnet":
        processed_img = preprocess_input_resnet50(img_batch)
    elif model_name == "densenet":
        processed_img = preprocess_input_densenet(img_batch)
    elif model_name == "mobilenet":
        processed_img = preprocess_input_mobilenet(img_batch)
    else:
        raise ValueError("Invalid model name")
    
    return processed_img

def show_preprocess_image(loaded_image, model_names):
    """
    Preprocesses a loaded image based on the specified model
    """
    processed_images = []
    for model_name in model_names:
        processed_img = preprocess_image(loaded_image, model_name)
        processed_images.append(processed_img)
    
    return processed_images


def image_predict(preprocessed_images, non_image_features, model):
    """
    Returns class probabilities for given preprocessed images and non-image features,
    based on the loaded model
    """
    predictions = model.predict([preprocessed_images, non_image_features])
    probabilities = np.round(predictions[0], 6)
    CLASSES = os.listdir('./images')
    class_probabilities = dict(zip(CLASSES, probabilities))
    
    return class_probabilities

In [44]:
test_data = pd.read_csv("./Augmentation/test_df_names.csv")
test_data

,region_ARM,region_BACK,region_CHEST,region_EAR,region_FACE,region_FOOT,region_FOREARM,region_HAND,region_LIP,region_NECK,...,has_piped_water,smoke,drink,gender,pesticide,hurt,diagnostic,img_id,image_path,patient
0,0,0,0,1,0,0,0,0,0,0,...,1,0,1,1,1,0,ACK,PAT_595_1142_142.png,./cancer/all_cancer_images/PAT_595_1142_142.png,ACK_Ethan
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,ACK,PAT_388_2452_548.png,./cancer/all_cancer_images/PAT_388_2452_548.png,ACK_Olivia
2,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,ACK,PAT_526_3205_611.png,./cancer/all_cancer_images/PAT_526_3205_611.png,ACK_Benjamin
3,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,ACK,PAT_851_1623_506.png,./cancer/all_cancer_images/PAT_851_1623_506.png,ACK_Emma
4,0,0,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,ACK,PAT_951_1836_991.png,./cancer/all_cancer_images/PAT_951_1836_991.png,ACK_Caleb
5,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,ACK,PAT_184_284_721.png,./cancer/all_cancer_images/PAT_184_284_721.png,ACK_Sophia
6,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,ACK,PAT_763_1439_668.png,./cancer/all_cancer_images/PAT_763_1439_668.png,ACK_Isaac
7,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,ACK,PAT_180_756_831.png,./cancer/all_cancer_images/PAT_180_756_831.png,ACK_Ava
8,0,0,0,1,0,0,0,0,0,0,...,0,1,0,1,1,1,ACK,PAT_224_1340_142.png,./cancer/all_cancer_images/PAT_224_1340_142.png,ACK_Matthew
9,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,0,1,ACK,PAT_492_1784_50.png,./cancer/all_cancer_images/PAT_492_1784_50.png,ACK_Mia


In [51]:
def main():
    # Load the model
    model = load_model("0.7164(new).h5")

    # Load the class labels
    CLASSES = {
        "ACK": "Actinic Keratosis",
        "MEL": "Malignant Melanoma",
        "BCC": "Basal Cell Carcinoma of skin",
        "SCC": "Squamous Cell Carcinoma"
    }

    # Load non-image features
    test_data = pd.read_csv("./Augmentation/test_df_names.csv")
    image_dir = './zr7vgbcyr2-1/all/test'

    # Initialize lists to store results
    predictions = []
    true_labels = []

    # Iterate over test data
    for index, row in test_data.iterrows():
        patient_name = row['patient']
        img_id = row['img_id']
        true_label = row['diagnostic']

        # Construct the image path
        image_path = os.path.join(image_dir, img_id)

        # Load and preprocess the image
        loaded_image = load_image(image_path)
        model_names = ["resnet", "densenet", "mobilenet"]
        processed_images = show_preprocess_image(loaded_image, model_names)

        # Get non-image features
        non_image_features = np.array(row.drop(["diagnostic", "img_id", "image_path", "patient"])).astype(float)  # Convert to float

        # Replicate non_image_features to match the batch size
        num_samples = processed_images[0].shape[0]
        non_image_features = np.tile(non_image_features, (num_samples, 1))

        # Make predictions on the image
        class_probabilities = image_predict(processed_images, non_image_features, model)

        # Append results to lists
        predictions.append(class_probabilities)
        true_labels.append(true_label)

    # Create a dataframe with predictions and true labels
    df_predictions = pd.DataFrame(predictions)
    df_predictions['True_Label'] = true_labels

    # Convert predictions to numeric values
    df_predictions = df_predictions.apply(pd.to_numeric, errors='coerce')

    # Map class labels to their names
    df_predictions['Predicted_Class'] = df_predictions.idxmax(axis=1).map(CLASSES.get)

    return df_predictions



In [52]:
df_results = main()
df_results

1/1 [==============================] - 2s 2s/step


AttributeError: 'dict' object has no attribute 'astype'

In [42]:
def accuracy_score(model, x, y):
    """
    This function calculates the multi-class accuracy score.
    """

    # Load the class labels
    CLASSES = {
        "ACK": "Actinic Keratosis",
        "MEL": "Malignant Melanoma",
        "BCC": "Basal Cell Carcinoma of skin",
        "SCC": "Squamous Cell Carcinoma",
        "SEK": "Seborrheic Keratosis",
        "NEV": "Nevus"
    }

    # Make predictions using the provided model
    predictions = main()

    # Group predictions by class and sum the probabilities
    grouped_df = predictions.groupby('Predicted_Class').sum()
    grouped_df.sort_index(axis=1, inplace=True)

    # Calculate the accuracy score
    accuracy = np.trace(grouped_df.values) / np.sum(grouped_df.values)
    accuracy = round(accuracy, 6)

    return accuracy


In [43]:
# Calculate accuracy score using the model, xtest, and ytest
#model = load_model("0.7164(new).h5")
accuracy = accuracy_score(model, xtest, ytest)
print("Accuracy Score:", accuracy)

1/1 [==============================] - 0s 44ms/step
Accuracy Score: 0.877355
